# Getting started with Cirq and IonQ on Azure Quantum

This notebook shows how to send a basic quantum circuit expressed using the [Cirq library](https://quantumai.google/cirq) to an IonQ target via the Azure Quantum service.

First, install `azure-quantum` with the Cirq dependencies:

In [1]:
# To install the Azure Quantum client package with support for
# Cirq, uncomment and run the line below:
#
# !pip install azure-quantum[cirq]==0.19.2109.165653 --quiet

## Connecting to the Azure Quantum service

To connect to the Azure Quantum service, find the resource ID and location of your Workspace from the Azure portal here: https://portal.azure.com. Navigate to your Azure Quantum workspace and copy the values from the header.

<img src="../azure-quantum-resource-id.png" alt="How to retrieve the resource ID and location from an Azure Quantum workspace">

Paste the values into the `AzureQuantumService` constructor below to create a `service` that connects to your Azure Quantum Workspace. Optionally, specify a default target:

In [2]:
from azure.quantum.cirq import AzureQuantumService
service = AzureQuantumService(
    resource_id="",
    location="",
    default_target="ionq.simulator"
)

### List all targets

You can now list all the targets that you have access to, including the current queue time and availability. Running this method will trigger authentication to your Microsoft account, if you're not already logged in.

In [3]:
service.targets()

[<Target name="ionq.qpu", avg. queue time=19 s, Available>,
 <Target name="ionq.simulator", avg. queue time=0 s, Available>,
 <Target name="quantinuum.qpu.h1-1", avg. queue time=0 s, Unavailable>,
 <Target name="quantinuum.sim.h1-1sc", avg. queue time=0 s, Available>,
 <Target name="quantinuum.qpu.h1-2", avg. queue time=93509 s, Available>,
 <Target name="quantinuum.sim.h1-2sc", avg. queue time=0 s, Available>,
 <Target name="quantinuum.sim.h1-1e", avg. queue time=26 s, Available>]

## Run a simple circuit

Let's create a simple Cirq circuit to run. This circuit uses the square root of X gate, native to the IonQ hardware system.

This notebook assumes some familiarity with Cirq. To read more about Cirq, review the [Cirq documentation](https://quantumai.google/cirq).

In [4]:
import cirq

q0, q1 = cirq.LineQubit.range(2)
circuit = cirq.Circuit(
    cirq.X(q0)**0.5,              # Square root of X
    cirq.CX(q0, q1),              # CNOT
    cirq.measure(q0, q1, key='b') # Measure both qubits
)
circuit

0: ───X^0.5───@───M('b')───
              │   │
1: ───────────X───M────────

You can now run the program via the Azure Quantum service and get the result. The following cell will submit a job that runs the circuit with 100 shots, wait until the job is completed and return the results.

In [5]:
result = service.run(program=circuit, repetitions=100)

This returns a `cirq.Result` object.

In [6]:
print(result)

b=0001101010110000111010001110010111010001101010001111100011100001111100101011101101001000001000011000, 0001101010110000111010001110010111010001101010001111100011100001111100101011101101001000001000011000


### Run on IonQ QPU

#### Note: The time required to run a circuit on the QPU may vary depending on current queue times.

The previous job ran on the default simulator we specified, `"ionq.simulator"`. To run on the QPU, provide `"ionq.qpu"` as the `target` argument:

In [7]:
result = service.run(
    program=circuit,
    repetitions=100,
    target="ionq.qpu",
    timeout_seconds=500 # Set timeout to 500 seconds to accomodate current queue time on QPU
)

Again, this returns a `cirq.Result` object.

In [8]:
print(result)

b=0101011011011111100001011101101011011110100010000000011110111000100100110110101100110001001111101111, 0101011011011111100001011101101011011110100010000000011110111000100100110110101100110001001111101111



## Asynchronous model using Jobs

For long-running circuits, it can be useful to run them asynchronously. The `service.create_job` method returns a `Job`, which you can use to get the results after the job has run successfully.

In [9]:
job = service.create_job(
    program=circuit,
    repetitions=100,
    target="ionq.simulator"
)

To check on the job status, use `job.status()`:

In [10]:
job.status()

'ready'

To wait for the job to be done and get the results, use the blocking call `job.results()`:

In [11]:
result = job.results()
print(result)

00: 0.5
11: 0.5


Note that this does not return a `cirq.Result` object. Instead it returns a result object that is specific to the IonQ simulator and uses state probabilities instead of shot data.

In [12]:
type(result)

cirq_ionq.results.SimulatorResult

To convert this to a `cirq.Result` object, use `result.to_cirq_result()`:

In [13]:
result.to_cirq_result()

b=1101000111010101101001011100001111101110011000100111101000010100101000001100010010011010111010011101, 1101000111010101101001011100001111101110011000100111101000010100101000001100010010011010111010011101

In [14]:
# Print package versions used in this notebook
import cirq
print('\n'.join(f'{m.__name__}=={m.__version__}' for m in globals().values() if getattr(m, '__version__', None)))

cirq==0.12.0
